In [1]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
from datetime import datetime
import time
from openai import OpenAI
import re
load_dotenv()

True

In [ ]:
# Pour récupérer les données des films : 
url = "https://api.themoviedb.org/3/discover/movie"

parameters = {
    "language" : "fr-FR",
    "include_adult" : False,
    "include_video" : False,
    "page" : 1,
    "primary_release_date.lte" : "2020-01-01",
    "primary_release_date.gte" : "2010-01-01",
    "vote_average.gte" : 5.0,
    "with_runtime.gte" : 45,
    "with_runtime.lte" : 240,
    "sort_by" : "popularity.desc",
}

headers = {
    "accept": "application/json",
    "Authorization": os.getenv("KEY")
}

df_movies = pd.DataFrame()

for year in range(1950, 2030, 10):
    parameters["primary_release_date.lte"] = f"{year + 9}-12-31"
    parameters["primary_release_date.gte"] = f"{year}-01-01"
    try:
        while(True):
            r = requests.get(url, headers=headers, params=parameters)
            df_new = pd.json_normalize(json.loads(r.text), record_path="results")
            df_movies = pd.concat([df_movies, df_new], ignore_index=True)
            parameters["page"] += 1
            if parameters["page"] % 40 == 0:
                time.sleep(1)
            print(f"\rProgress: {parameters["page"]}, {parameters["primary_release_date.lte"]}", end='', flush=True)
    except Exception as e:
        if r.status_code == 400:
            parameters["page"] = 1
        else:
            print(r.status_code)
            print(f"Une erreur s'est produite : {e}")
            break

In [2]:
df = pd.read_parquet("movies.parquet") 
print(df.shape)
df.head(3)

(77770, 14)


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,"[10751, 14, 16, 10749]",11224,en,Cinderella,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,False,7.041,6685
1,False,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,[18],389,en,12 Angry Men,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,False,8.500,8655
2,False,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,"[18, 36]",6844,en,The Ten Commandments,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,False,7.756,1603


In [6]:
# Pour supprimer les doublons et ne garder que les vote count >= 500
df = df.drop_duplicates(subset=['id'], ignore_index=True)
df = df[df['vote_count'] >= 500]

In [ ]:
# API Catégorie genre
url = "https://api.themoviedb.org/3/genre/movie/list"

headers = {
    "accept": "application/json",
    "Authorization": os.getenv("tmdb_key")
}

response = requests.get(url, headers=headers)
print(response.text) # response tout seul envoie le statut 200
data = json.loads(response.text)
df_genre = pd.json_normalize(data, record_path='genres')

In [7]:
# Création d'un dictionnaire avec les genres : 
df_genre = pd.read_parquet("genre.parquet")
df_genre = df_genre.set_index('id')
genre_string = df_genre.to_json()
genre_dict = json.loads(genre_string)
genre_dict['name']

{'28': 'Action',
 '12': 'Adventure',
 '16': 'Animation',
 '35': 'Comedy',
 '80': 'Crime',
 '99': 'Documentary',
 '18': 'Drama',
 '10751': 'Family',
 '14': 'Fantasy',
 '36': 'History',
 '27': 'Horror',
 '10402': 'Music',
 '9648': 'Mystery',
 '10749': 'Romance',
 '878': 'Science Fiction',
 '10770': 'TV Movie',
 '53': 'Thriller',
 '10752': 'War',
 '37': 'Western'}

In [8]:
# replacer les id des genres dans le df_movie par la liste des genres
def str_genre(list_genre: list, genre_dict: dict)-> str:
    empty_list = []
    for element in list_genre : 
        element = str(element)
        if element in genre_dict: 
            empty_list.append(genre_dict[element])
    return empty_list

In [9]:
# on applique la fonction et on crée une nouvelle colonne 'genre'
df['genre'] = df['genre_ids'].apply(lambda x: str_genre(x, genre_dict['name']))
df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre
0,False,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,"[10751, 14, 16, 10749]",11224,en,Cinderella,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,False,7.041,6685,"[Family, Fantasy, Animation, Romance]"
1,False,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,[18],389,en,12 Angry Men,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,False,8.500,8655,[Drama]
2,False,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,"[18, 36]",6844,en,The Ten Commandments,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,False,7.756,1603,"[Drama, History]"
3,False,/sU3gFo7Gn67zjtKXIvzJ4omcUbH.jpg,"[36, 18, 12, 28]",665,en,Ben-Hur,"Judas Ben‐Hur, prince de Judée, retrouve son a...",56.369,/4KH7VM5YyGwxe4kLSNxix2PNTn6.jpg,1959-11-18,Ben-Hur,False,7.881,2747,"[History, Drama, Adventure, Action]"
4,False,/1HWNGfaYB4GPwMg8piXYdQHWNuF.jpg,"[14, 16, 10749, 10751]",10882,en,Sleeping Beauty,La sorcière Maléfique jette un sort à la princ...,46.057,/5GaczvmuUCxhwPNx896lo2OCIFh.jpg,1959-02-17,La Belle au bois dormant,False,6.900,5123,"[Fantasy, Animation, Romance, Family]"


In [10]:
# la fonction pour détécter tous les titres qui ne sont pas en lettres latines
def isascii(title: str) ->str: 
    car_special = ['à', 'â', 'ä', 'é', 'è', 'ê', 'ë', 'î', 'ï', 'ô', 'ö', 'ù', 'û', 'ü', 'ç']
    if title.isascii() or any(char in title for char in car_special):
        return title
    else:
        return 0

In [ ]:
# on applique la fonction et on crée une nouvelle colonne 'isascii'
df['isascii'] = df['title'].apply(isascii)

# on supprime tout ce qui n'est pas ascii et on drop la colonne 'isascii'
df = df[df["isascii"] != 0]
df.drop(columns=['isascii'], inplace=True)

In [37]:
df = df.reset_index()

In [ ]:
# API récupérer df_cast
df_cast = pd.DataFrame()
i = 0
for movie_id in df["id"]:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?"
    i += 1
    headers = {
        "accept": "application/json",
        "Authorization": "tmdb_key"
    }
    r = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(r.text), record_path="cast")
    df_new["movie_id"] = movie_id
    if df_cast.empty:
        df_cast = df_new
    else:
        df_cast = pd.concat([df_cast, df_new], ignore_index=True)
    if i % 40:
        time.sleep(1)
    print(f"\rProgress: {i} on 5772", end='', flush=True)

df_cast.to_parquet("df_cast.parquet")
df_actor = pd.read_parquet("df_cast.parquet")
df_actor.head()

In [ ]:
# dans df_actor ne garder que les ordres 1-4 et que le departement Acting
df_actor = df_actor[df_actor['order'] <= 4]

df_actor = df_actor[df_actor['known_for_department'] =='Acting']


In [ ]:
# API pour récupérer df_crew ()
df_crew = pd.DataFrame()
i = 0
for movie_id in df["id"]:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?"
    i += 1
    headers = {
        "accept": "application/json",
        "Authorization": "tmdb_key"
    }
    r = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(r.text), record_path="crew")
    df_new["movie_id"] = movie_id
    if df_crew.empty:
        df_crew = df_new
    else:
        df_crew = pd.concat([df_crew, df_new], ignore_index=True)
    if i % 40:
        time.sleep(1)
    print(f"\rProgress: {i} on 5772", end='', flush=True)
df_crew.head()

In [ ]:
# Dans df_crew garder uniquement le job = Director
df_crew.to_parquet("df_crew.parquet")

df_director = pd.read_parquet("df_crew.parquet")
df_director = df_director[df_director['job'] == 'Director']
df_director.head()

In [ ]:
# API people : le lien pour récupérer le détail des réalisateurs :  
df_director_details = pd.DataFrame()
count  = 0
for id in df_director['id_director']:
    url = f"https://api.themoviedb.org/3/person/{id}?language=fr-FR'"

    headers = {
    "accept": "application/json",
    "Authorization": os.getenv("tmdb_key")}

    response = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(response.text))
    df_director_details = pd.concat([df_director_details, df_new], ignore_index=True)
    count += 1
    if count == 100:
        time.sleep(3)
        count = 0
df_director_details.to_parquet("df_director_details.parquet")

In [ ]:
# PEOPLE : le lien pour récupérer le détail des acteurs :
df_actors_details = pd.DataFrame()
count  = 0
for id in df_actor['id_actor']:
    url = f"https://api.themoviedb.org/3/person/{id}?language=fr-FR'"

    headers = {
    "accept": "application/json",
    "Authorization": os.getenv("tmdb_key")}

    response = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(response.text))
    df_director_details = pd.concat([df_director_details, df_new], ignore_index=True)
    count += 1
    if count == 100:
        time.sleep(3)
        count = 0
df_actors_details.to_parquet("df_actors_details.parquet")

In [ ]:
# Jointure des tables :
df_director = pd.merge(df, df_director, how= left, left_on="id", right_on="id_director")
df_actors_final = pd.merge(df_actor, df_actors_details, how=left, on = ["id", "id"] )
df = pd.merge(df, df_director, how= left, on = ["id_movie", "id_movie"])


In [ ]:
# Rajouter les colonnes acteur_1 - acteur_5 dans le dataframe des movies et directeurs: 

df_actors_final[f"name_actor_1"] = None

for movie_id in df_actors_final["movie_id"]:
    df_temp = df_actor[df_actor["movie_id"] == movie_id] 
    df_temp = df_temp.sort_values(by="order").reset_index()
    try :
        df_actors_final.loc[df_actors_final['movie_id'] == movie_id, f'name_actor_1'] = df_temp.loc[1, "name"]
    except:
        df_actors_final.loc[df_actors_final['movie_id'] == movie_id, f'name_actor_1'] = None
df_actors_final.sort_values(by=['movie_id'])

In [38]:
df1 = df.loc[0 : 2887]
df2 = df.loc[2887 : ]

In [40]:
data1 = pd.DataFrame(columns=['id_movie', 'Publique cible', 'Epoque', 'Sujet', ])
data1['id_movie'] = df1['id']
data1

,id_movie,Publique cible,Epoque,Sujet
0,11224,NaN,NaN,NaN
1,389,NaN,NaN,NaN
2,6844,NaN,NaN,NaN
3,665,NaN,NaN,NaN
4,10882,NaN,NaN,NaN
...,...,...,...,...
2883,138843,NaN,NaN,NaN
2884,76341,NaN,NaN,NaN
2885,337167,NaN,NaN,NaN
2886,273481,NaN,NaN,NaN


In [41]:
data2 = pd.DataFrame(columns=['id_movie', 'Publique cible', 'Epoque', 'Sujet', ])
data2['id_movie'] = df2['id']
data2

,id_movie,Publique cible,Epoque,Sujet
2887,396535,NaN,NaN,NaN
2888,73723,NaN,NaN,NaN
2889,315635,NaN,NaN,NaN
2890,210577,NaN,NaN,NaN
2891,273477,NaN,NaN,NaN
...,...,...,...,...
5767,694234,NaN,NaN,NaN
5768,714888,NaN,NaN,NaN
5769,606954,NaN,NaN,NaN
5770,614292,NaN,NaN,NaN


In [8]:
# OPEN AI : pour récupérer les mots-cléfs par film: 

cat_list = ['Publique cible','Epoque', 'Sujet principal', 'Type']

XAI_API_KEY = os.getenv("open_ai_key_noame")

client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)
count = 0
for i in range(0, len(df1)): 
    completion = client.chat.completions.create(
        model="grok-2-1212",
        messages=[
            {"role": "system", "content": "Tu es cinéphile, tu connais tous les films depuis les années 1950."},
            {"role": "user", "content": f"""Pour le film {df1.loc[i, 'title']} sorti en {df1.loc[i, 'release_date']} choisit un seul mot  par catégorie parmi ces choix: 
            1) Publique cible : enfant, adulte; 
            2) Epoque de l'intrigue : préhistoire, antiquité, moyen-âge, classique, année 1950-1960,  années 1970-1980, années 1990 -2000,  
            époque actuelle,  futuriste; 
            3) Sujet principal : amour, espace, guerre, super-héros, peur, postapocalyptique, 
            catastrophe naturelle, science, humour, animaux, biopic, voyage dans le temps, polar, drame, suspense. 
            4) Type : dessin animé, blockbuster, film d'auteur, film français, autre. 
            Ne prends aucune initiative, ne donne qu'un seul mot par catégorie. 
            Donne moi les données sous cette forme : [1, 2, 3, 4]"""},
        ],
    )

    #completion : Représente la réponse de l'API.
    #choices[0] : Prend la première réponse générée (souvent, l'API peut produire plusieurs réponses si configurée pour le faire).
    #message.content : Contient le texte généré par l'API, ici les 5 mots-clés décrivant Cendrillon.*

    id_movie = df1.loc[i, 'id_movie']
    response = completion.choices[0].message.content
    response = response.replace('[', "").replace(']', "").split(', ')
    empty_dict = {id_movie: response}
    for idx, category in enumerate(cat_list): 
        data1.loc[i, category] = empty_dict[id_movie][idx]
    
    time.sleep(4)
    
data1.to_parquet("save1.parquet")

In [5]:
# OPEN AI 

cat_list = ['Publique cible','Epoque', 'Sujet principal', 'Type']

XAI_API_KEY = os.getenv("open_ai_key_manon")

client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)


for i in range(0, len(df2)): 
    completion = client.chat.completions.create(
        model="grok-2-1212",
        messages=[
            {"role": "system", "content": "Tu es cinéphile, tu connais tous les films depuis les années 1950."},
            {"role": "user", "content": f"""Pour le film {df2.loc[i, 'title']} sorti en {df2.loc[i, 'release_date']} choisit un seul mot  par catégorie parmi ces choix: 
            1) Publique cible : enfant, adulte; 
            2) Epoque de l'intrigue : préhistoire, antiquité, moyen-âge, classique, année 1950-1960,  années 1970-1980, années 1990 -2000,  
            époque actuelle,  futuriste; 
            3) Sujet principal : amour, espace, guerre, super-héros, peur, postapocalyptique, 
            catastrophe naturelle, science, humour, animaux, biopic, voyage dans le temps, polar, drame, suspense. 
            4) Type : dessin animé, blockbuster, film d'auteur, film français, autre. 
            Ne prends aucune initiative, ne donne qu'un seul mot par catégorie. 
            Donne moi les données sous cette forme : [1, 2, 3, 4]"""},
        ],
    )

    #completion : Représente la réponse de l'API.
    #choices[0] : Prend la première réponse générée (souvent, l'API peut produire plusieurs réponses si configurée pour le faire).
    #message.content : Contient le texte généré par l'API

    id_movie = df2.loc[i, 'id_movie']
    response = completion.choices[0].message.content
    response = response.replace('[', "").replace(']', "").split(', ')
    empty_dict = {id_movie: response}
    for idx, category in enumerate(cat_list): 
        data2.loc[i, category] = empty_dict[id_movie][idx]
    
    time.sleep(4)

data2.to_parquet("save2.parquet")

In [2]:
save1 = pd.read_parquet("save1.parquet")
save1

,id_movie,Publique cible,Epoque,Sujet principal,Type
0,11224,enfant,moyen-âge,amour,dessin animé
1,389,adulte,année 1950-1960,drame,film d'auteur
2,6844,adulte,antiquité,drame,blockbuster
3,665,adulte,antiquité,drame,blockbuster
4,10882,enfant,moyen-âge,amour,dessin animé
...,...,...,...,...,...
2882,138843,adulte,années 1970-1980,peur,blockbuster
2883,76341,adulte,futuriste,postapocalyptique,blockbuster
2884,337167,adulte,époque actuelle,amour,blockbuster
2885,273481,adulte,époque actuelle,suspense,blockbuster


In [6]:
save2 = pd.read_parquet("save2.parquet")
save2

,id_movie,Publique cible,Epoque,Sujet principal,Type
0,73723,enfant,futuriste,animaux,dessin animé
1,315635,adulte,époque actuelle,super-héros,blockbuster
2,210577,adulte,époque actuelle,suspense,blockbuster
3,273477,adulte,futuriste,postapocalyptique,blockbuster
4,10192,enfant,époque actuelle,humour,dessin animé
...,...,...,...,...,...
2879,694234,adulte,époque actuelle,amour,film d'auteur
2880,714888,adulte,années 1970-1980,biopic,film d'auteur
2881,606954,adulte,époque actuelle,drame,film d'auteur
2882,614292,adulte,époque actuelle,drame,film d'auteur


In [ ]:
# Concater les deux DF save
save = pd.concat([save1, save2])
df = pd.merge(df1, save, how='left', on = ["id_movie", "id_movie"])
df_movie = df.copy()

In [ ]:
# Remplacer les dates par les décennies
date_list = ['release_date', 'birthday_director', 'deathday_director']
for date in date_list:
    try:
        df_movie[date] = pd.to_datetime(df_movie[date])
        df_movie[date] = ((df_movie[date].dt.year // 10) * 10)
    except: 
        pass

In [ ]:
# remplir les valeurs nulls
df_movie[['release_date', 'birthday_director', 'deathday_director']] = df_movie[['release_date', 
                                                                                 'birthday_director', 'deathday_director']].fillna(0)

In [ ]:
# Modifier le type en int
df_movie['birthday_director'] = df_movie['birthday_director'].astype(int)
df_movie['deathday_director'] = df_movie['deathday_director'].astype(int)
df_movie.head(3)

In [ ]:
# Remplir les valeurs nulles dans d'autres colonnes non-numériques : 
df_movie[['actor_name_2', 
          'actor_name_3', 'actor_name_4', 'actor_name_5']] = df_movie[['actor_name_2', 
                                                                       'actor_name_3', 'actor_name_4', 'actor_name_5']].fillna('aucun')

In [ ]:
# Créer un fichier parquet avec les colonnes nécéssaires pour la ML
df_movie = df_movie[['id_movie','original_language', 'popularity_movie', 'release_date', 'title',
       'vote_average', 'vote_count', 'genre', 'name_director', 'popularity_director',
       'actor_name_1', 'actor_name_2', 'actor_name_3', 'actor_name_4',
       'actor_name_5', 'Publique cible', 'Epoque', 'Sujet principal', 'Type']]

In [ ]:
df_movie.to_parquet("df_movie.parquet")